In [2]:
import pandas as pd

file1 = 'CSV/features.csv'
file2 = 'CSV/train.csv'

full_table1 = pd.read_csv(file1)
full_table2 = pd.read_csv(file2)

print(full_table1.columns)
print(full_table2.columns)

Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday'],
      dtype='object')
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')


In [3]:
#Merging tables
merged_data = pd.merge(full_table1, full_table2, on=["Store", "Date", "IsHoliday"])
print(merged_data.head())

   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2   
0      1  05/02/2010        42.31       2.572        NaN        NaN  \
1      1  05/02/2010        42.31       2.572        NaN        NaN   
2      1  05/02/2010        42.31       2.572        NaN        NaN   
3      1  05/02/2010        42.31       2.572        NaN        NaN   
4      1  05/02/2010        42.31       2.572        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  Dept   
0        NaN        NaN        NaN  211.096358         8.106      False     1  \
1        NaN        NaN        NaN  211.096358         8.106      False     2   
2        NaN        NaN        NaN  211.096358         8.106      False     3   
3        NaN        NaN        NaN  211.096358         8.106      False     4   
4        NaN        NaN        NaN  211.096358         8.106      False     5   

   Weekly_Sales  
0      24924.50  
1      50605.27  
2      13740.12  
3      39954.0

In [4]:
#Creating a new data frame with only desired variables
desired_vars = {
    "Date":merged_data['Date'],
    "Weekly_Sales": merged_data["Weekly_Sales"],
    "Temperature": merged_data["Temperature"],
    "Fuel_Price": merged_data["Fuel_Price"],
    "IsHoliday": merged_data["IsHoliday"]
}

data_table = pd.DataFrame(desired_vars)
print(data_table.head())

         Date  Weekly_Sales  Temperature  Fuel_Price  IsHoliday
0  05/02/2010      24924.50        42.31       2.572      False
1  05/02/2010      50605.27        42.31       2.572      False
2  05/02/2010      13740.12        42.31       2.572      False
3  05/02/2010      39954.04        42.31       2.572      False
4  05/02/2010      32229.38        42.31       2.572      False


In [5]:
#Splitting data
train_data = data_table[data_table["Date"] < "2012-01-01"]
test_data = data_table[data_table["Date"] >= "2012-01-01"]

In [8]:
#Creation of the SARIMAX model

import statsmodels.api as sm

#Convert IsHoliday bool type to int
train_data["IsHoliday"] = train_data["IsHoliday"].astype(int)

# Define exogenous variables
exog_vars = ["Temperature", "Fuel_Price", "IsHoliday"]

# Create SARIMAX model
model = sm.tsa.SARIMAX(train_data["Weekly_Sales"], exog=train_data[exog_vars],
                       order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the model
results = model.fit()

# Print the model summary
print(results.summary())

C:\Users\User\AppData\Local\Temp\ipykernel_3804\2772326072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["IsHoliday"] = train_data["IsHoliday"].astype(int)
c:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                                     SARIMAX Results                                      
Dep. Variable:                       Weekly_Sales   No. Observations:               280198
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood            -3207921.857
Date:                            Tue, 23 May 2023   AIC                        6415859.714
Time:                                    00:20:31   BIC                        6415944.059
Sample:                                         0   HQIC                       6415884.181
                                         - 280198                                         
Covariance Type:                              opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Temperature   -57.3380     16.425     -3.491      0.000     -89.531     -25.145
Fuel_Price   5690.2425    853.12

In [ ]:
#Making predioctions
#Forecast future values
forecast = results.get_forecast(steps=len(test_data), exog=test_data[exog_vars])

# Extract the predicted values
predicted_values = forecast.predicted_mean

In [ ]:
#Evaluating Model
#Compare predicted values with actual values
mse = ((predicted_values - test_data["Weekly_Sales"]) ** 2).mean()